<center>

## MAGICIAN: Mining and gAmblinG servIces sCrapIng and ANalysis

Matteo Martini 636694

In [77]:
mining_pool_url = "https://en.bitcoin.it/wiki/DeepBit9"
full_wallet_address = "https://www.walletexplorer.com/wallet/"


#Individuare nel DataSet tutte le transazioni relative a Deepbit.net,ovvero le transazioni che presentano almeno un indirizzo di input o di output appartenente a Deepbit_addrs.
#servizio scommesse = DiceOnCrack

## Setup Ambiente

In [78]:
# Librerie concesse
import pandas as pd

# Per scraping (scegline uno o entrambi, in base alle esigenze delle pagine)
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Per formati efficienti (PyArrow)
import pyarrow as pa
import pyarrow.parquet as pq

# Utility
from pathlib import Path
from datetime import datetime

import numpy as np


from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import time
import requests
import re
from bs4 import BeautifulSoup

# Percorsi (aggiorna con il tuo percorso locale)
DATA_DIR = Path('./Datasets')   # metti qui i CSV scaricati

PARQUET_DIR = Path('./data_parquet')
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
print('Environment ready.')

Environment ready.


## 1) Costruzione dei Dataframe dal Dataset

## Directory Datasets


In [79]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

## Definizone Data Type
Sapendo cosa contengono i file csv, possiamo forzare la conversione in determiinati tipi per ridurre l'impatto in memoria dei dataset

In [119]:
dtype_transactions = {
    "timestamp": np.int32,          # prima colonna: secondi (1231469xxx) rientra in int32
    "block_height": np.int32,              # seconda colonna: contatore 1…10 572 827
    "tx_id": np.int32,       # terza colonna: 1…214 562
    "prev_tx_id": np.int32,         # quarta colonna: seguono il tx_id precedente
    "is_coinbase": np.uint8,        # quinta colonna 0/1
    "fee": np.int32,                # sesta colonna: 0…100 000 (int32 più che sufficiente)
}


dtype_inputs = {
    "input_id": np.int32,
    "tx_id": np.int32,
    "prev_output_id": np.int32,
    "input_index": np.uint8,
}


dtype_outputs = {
    "col0":np.int32,
    "col1":np.int32,
    "col2":np.int8,
    "col3":np.int32,
    "col4":np.int64,
    "col5":"uint8"
}

dtype_mapping = {
    "address": "category",   # stringhe ripetute → category riduce drasticamente
    "address_id": "Int32",   # valori 0–8 708 820 entrano in 32 bit
}


columns_inputs = [
    "tx_id",
    "prev_tx_id",
    "prev_tx_pos"
]
columns_transactions = [
    "timestamp", "block_height", "tx_id",
     "is_coinbase", "fee"
]
columns_outputs = [
    "tx_id","position",
    "addressId","amount","script_type"
]
columns_mapping = [
    "hash", "addressId"
]

## Caricamento dei Dataset

In [120]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

transactions = pd.read_csv(transactions_csv,header=None,names=columns_transactions,dtype=dtype_transactions)
inputs_df    = pd.read_csv(inputs_csv,header=None,names=columns_inputs,dtype=dtype_inputs)
outputs_df   = pd.read_csv(outputs_csv,header=None,names=columns_outputs,dtype=dtype_outputs)
mapping_df   = pd.read_csv(mapping_csv,header=None,names=columns_mapping,dtype=dtype_mapping)

## Operazioni su Dataset

In [82]:
merged_outputs = outputs_df.merge(mapping_df,on="addressId",how="left")
display(merged_outputs)

,tx_id,position,addressId,amount,script_type,hash
0,0,0,0,5000000000,1,1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa
1,1,0,1,5000000000,1,12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX
2,2,0,2,5000000000,1,1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1
3,3,0,3,5000000000,1,1FvzCLoTPGANNjWoUo6jUGuAG3wg1w4YjR
4,4,0,4,5000000000,1,15ubicBBWFnvoZLT7GiU2qxjRaKJPdkDMG
...,...,...,...,...,...,...
24613794,10572825,0,8707915,1,2,1DYve98WqJQmx6cDdW2EQFpnqfwptjV4e9
24613795,10572825,1,6137715,9899999,2,1AdN2my8NxvGcisPGYeQTAKdWJuUzNkQxG
24613796,10572826,0,8707915,26651306,2,1DYve98WqJQmx6cDdW2EQFpnqfwptjV4e9
24613797,10572826,1,6137711,7991620447,2,1L4EThM6x3Rd2PjNbs1U136FpMq4Gmo3fJ


## 2) Recupero informazioni da WalletExplorer


### Definizione Funzioni Helper per Scraping

In [83]:
wallet_explorer_url = "https://www.walletexplorer.com/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/127.0.0.1 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.walletexplorer.com/",
}

In [84]:
def next_page(driver):
    #cerco il link alla prossima pagina
    for _ in ("Next","Last"):
        try:
            el = driver.find_element(By.XPATH, f'//div[@class="paging"]//a[contains(., "{_}")]')
            url = el.get_attribute("href")
            el.click()
            return url
        except:
            return None
        
def find_next_link(driver,element):
    try:
        el = driver.find_element(By.XPATH, f'//span[@class="showother"]//a[contains(., "{element}")]')
        url = el.get_attribute("href")
        el.click()
        return url
    except:
        return None

# def get_transactions_link(soup,class_):
#     transactions = soup.find_all('tr',class_=class_)
#     page_links = []
#     for trans in transactions:
#         txid = trans.find('td',class_="txid").find('a')['href']
#         page_links.append(txid)
#     return page_links

# def safe_get(session, url, tries=3, delay=1):
    for attempt in range(tries):
        try:
            return session.get(url, timeout=10)
        except (requests.exceptions.ConnectionError,
                requests.exceptions.RemoteDisconnectedError):
            if attempt == tries - 1:
                raise
            time.sleep(delay * (attempt + 1))  # 1s, poi 2s, poi 3s…

def scrape_step(driver):
    #recupero le entries
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # seleziona tutti gli elementi che hanno l’attributo data-wallet-id
    wallet_elements = soup.select("td a[href^='/address/']") 
    # estrai il valore dell’attributo
    return [re.search(r"/address/([^/?#]+)", w.get("href")).group(1) for w in wallet_elements if w.get("href") and re.search(r"/address/([^/?#]+)", w.get("href"))]




### Definizione Funzione per Scraping

In [85]:
def scrape_wallet_explorer(wallet_name):
    driver = webdriver.Chrome()
    driver.get(wallet_explorer_url)
    
    #ricerca del wallet tramite form
    driver.find_element(By.NAME,'q').send_keys(wallet_name)
    driver.find_element(By.TAG_NAME,'form').submit()
    
    
    #mettere funzione di selenium
    time.sleep(1)
    
    #mostra wallet
    tri = find_next_link(driver,"show wallet addresses")
    if tri == None: return
    #mettere funzione di selenium
    time.sleep(1)

    try:
        while True:
            #recupero di tutti gli input/output della pagina
            yield from scrape_step(driver)

            if next_page(driver) == None:break;
    except:
            print('Something went wrong')
    finally:
        driver.close()    
   
#https://www.walletexplorer.com/wallet/


<center>

# Analisi Transazioni DiceOnCrack


In [86]:
diceoncrack_transaction = set(scrape_wallet_explorer('DiceonCrack'))
print(len(diceoncrack_transaction))

28


<center>

# Analisi Transazioni di DeepBit.Net

In [125]:
deepbit_transactions = set(scrape_wallet_explorer('DeepBit.net'))
print(len(deepbit_transactions))

2


### Recupero gli output appartenenti a Deepbit.net

In [156]:
# display(merged_outputs)
bnet_out = merged_outputs[merged_outputs["hash"].isin(deepbit_transactions)]
deepbit_out_transactions = transactions[transactions["tx_id"].isin(bnet_out["tx_id"].unique())]
assert len(deepbit_out_transactions) > 0

### Recupero le transazioni Coinbase

In [178]:

# transazioni che inviano a Deepbit     
deepbit_trans_id = deepbit_out_transactions["tx_id"].unique()
# display(deepbit_trans_id)

deepbit_inputs = inputs_df[inputs_df["tx_id"].isin(deepbit_trans_id)]
# display(deepbit_inputs)

coinbase_deepb = transactions[transactions["tx_id"].isin(deepbit_inputs["prev_tx_id"].tolist()) & transactions["is_coinbase"] == 1]

In [ ]:
display(coinbase_deepb)

,timestamp,block_height,tx_id,is_coinbase,fee
1878757,1321605478,153792,1878755,1,0
1878902,1321606898,153795,1878900,1,0
1879039,1321607660,153798,1879037,1,0
1879139,1321611188,153800,1879137,1,0
1879203,1321611674,153801,1879201,1,0
...,...,...,...,...,...
10548297,1356924331,214422,10548295,1,0
10551605,1356932292,214442,10551603,1,0
10551691,1356932699,214443,10551689,1,0
10551821,1356932625,214444,10551819,1,0


### Recupero gli input appartenenti a Deepbit.net